<h2> Sistema de Recomendación de lugares para comer

<h5>En este script, vamos a mostrar cómo generar recomendaciones de tiendas basadas en características como la ubicación, categoría, reseñas y calificaciones.<br>
Este flujo de trabajo genera recomendaciones de tiendas basadas en características específicas utilizando un modelo de regresión lineal y representaciones vectoriales de reseñas procesadas por un modelo de lenguaje preentrenado.</h5>

<h3>Importación de bibliotecas</h3>
<h5>En esta sección, se importan las bibliotecas necesarias, como pandas para el manejo de datos, numpy para cálculos numéricos y LinearRegression de scikit-learn para el modelo de regresión lineal. También se importa el módulo math para cálculos trigonométricos.</h5>

In [93]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

<h3>Carga de datos</h3>
<h5>En esta parte, se cargan los conjuntos de datos desde archivos CSV utilizando la función read_csv de Pandas. Los datos incluyen información sobre categorías, tiendas, hoteles y puntuaciones de análisis de texto obtenidos de RoBERTa.</h5>

In [94]:
# Cargar los datos
df_category = pd.read_csv('C:/Users/PC/Dropbox/Henry/DATA-FT-12/Lab 03 - PF/DiagramaER/category.csv',sep=';')  
df_category_shop = pd.read_csv('C:/Users/PC/Dropbox/Henry/DATA-FT-12/Lab 03 - PF/DiagramaER/category_shop.csv',sep=';')
df_shop = pd.read_csv('C:/Users/PC/Dropbox/Henry/DATA-FT-12/Lab 03 - PF/ML/ML_shops.csv',sep=';') 
df_hotel = pd.read_csv('C:/Users/PC/Dropbox/Henry/DATA-FT-12/Lab 03 - PF/ML/ML_Hilton_hotels.csv',sep=';') 
df_RoBERTa = pd.read_csv('C:/Users/PC/Dropbox/Henry/DATA-FT-12/Lab 03 - PF/ML/roberta_Reviews_CA.csv',sep=';')

C:\Users\PC\AppData\Local\Temp\ipykernel_30300\624089931.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_RoBERTa = pd.read_csv('C:/Users/PC/Dropbox/Henry/DATA-FT-12/Lab 03 - PF/ML/roberta_Reviews_CA.csv',sep=';')


<h3>Datos de entrada generados por el Usuario</h3>
<h5>Aquí se selecciona aleatoriamente un hotel en California (hotel_ca) y se extrae su ubicación (latitud y longitud). También se definen las categorías seleccionadas por el cliente en la lista selected_categories.

In [95]:
# Datos de entrada
hotel_ca = df_hotel[df_hotel['state'] == 'CA'].sample(n=1)  # Selecciona un hotel al azar en California

lat1 = hotel_ca['latitude'].values[0] # Latitud de la ubicación
lon1 = hotel_ca['longitude'].values[0] # Longitud de la ubicación
user_location = (lat1, lon1)
selected_categories = [1, 3, 5]  # Categorías seleccionadas por el cliente

<h3>Cálculo de la región de interés para la búsqueda de los lugares</h3>
<h5>Se define una función calculate_distance que toma las coordenadas de dos ubicaciones y calcula la distancia geográfica entre ellas utilizando la fórmula de la ley de los cosenos. El resultado se devuelve en kilómetros.</h5>

In [96]:
# Cálculos para la distancia geográfica
def calculate_distance(lat1, lon1, lat2, lon2):

    radius = 6371  # Radio de la Tierra en kilómetros

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)

    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance

<h3>Filtrado de Locales</h3>
<h5>Filtramos los locales de comida según las categorías y la ubicación del usuario.<br>Se crea una lista filtered_shops que contiene las tiendas que están a una distancia de 10 km o menos de la ubicación del usuario.Luego se crea un DataFrame filtered_shops_df a partir de la lista de tiendas filtradas.<br>
A continuación, se seleccionan los identificadores de tiendas (id_shop) del DataFrame df_category_shop que tengan categorías presentes en la lista selected_categories. Se filtran las tiendas en el DataFrame filtered_shops_df para incluir solo aquellas que tienen identificadores de tiendas presentes en selected_id_shops.
</h5>

In [97]:
# Filtrar tiendas por distancia
filtered_shops = [
    shop for _, shop in df_shop.iterrows() if 
    calculate_distance(user_location[0], user_location[1], shop['latitude'], shop['longitude']) <= 10
]

# Crear DataFrame a partir de las tiendas filtradas
filtered_shops_df = pd.DataFrame(filtered_shops)

# Seleccionar id_shop en df_category_shop que tengan los valores de selected_categories
selected_id_shops = df_category_shop[df_category_shop['id_category'].isin(selected_categories)]['id_shop']

# Filtrar filtered_shops con los id_shop seleccionados
filtered_shops_df = filtered_shops_df[filtered_shops_df['id_shop'].isin(selected_id_shops)]
# filtered_shops = filtered_shops[filtered_shops['id_category'].isin(selected_categories)]

<h3>Empleo de las reseñas</h3>
<h5>Se crea un DataFrame shop_scores que contiene las puntuaciones RoBERTa y estrellas para las tiendas cuyos identificadores estén presentes en filtered_shops_df.<br>A continuación, Se calcula la mediana de las puntuaciones RoBERTa positivas (roberta_pos) y las puntuaciones de estrellas (stars) agrupadas por el identificador de fuente (id_source), lo que proporciona una estimación de las puntuaciones medianas para cada tienda.

In [98]:
# Obtener los valores RoBERTa_score y stars para cada tienda
shop_scores = df_RoBERTa[df_RoBERTa['id_source'].isin(filtered_shops_df['id_source'])]

# Calcular la mediana del campo roberta_pos y stars agrupados por id_source
median_scores = shop_scores.groupby('id_source')[['roberta_pos', 'stars']].median()

<h3>Definición del modelo, entrenamiento y recomendación.</h3>
<h5>En esta última sección, se define la función calculate_weighted_score que calcula un puntaje ponderado para cada tienda basado en las puntuaciones RoBERTa y estrellas.<br> Luego, se crea una matriz de características X y un vector objetivo y para el modelo de regresión lineal. Se itera a través de las tiendas filtradas, se calculan los puntajes ponderados y se construyen las matrices X e y.<br> Luego, se entrena el modelo de regresión lineal utilizando LinearRegression() y se generan recomendaciones ordenadas por puntaje ponderado. Finalmente, se imprimen las recomendaciones en la consola junto con el nombre de la tienda correspondiente.

In [91]:
def calculate_weighted_score(shop):
    # Obtener el id_source de la tienda
    shop_id = shop['id_source']
    
    # Buscar el puntaje roberta_pos y stars en shop_scores
    if shop_id in shop_scores['id_source'].values:
        shop_row = shop_scores[shop_scores['id_source'] == shop_id].iloc[0]
        return 0.6 * shop_row["roberta_pos"] + 0.4 * shop_row["stars"]
    else:
        return 0.0  # Manejar el caso si no se encuentra la tienda en shop_scores

# Crear matriz de características 'X' y vector objetivo 'y'
X = []
y = []

for shop in filtered_shops:
    shop_id = shop['id_source']
    if shop_id in median_scores.index:
        X.append([calculate_weighted_score(shop)])
        y.append(median_scores.loc[shop_id, ['roberta_pos', 'stars']].values)

X = np.array(X)
y = np.array(y)

# Entrenar el modelo de regresión lineal (nuevo código)
model = LinearRegression()
model.fit(X, y)

# Generar recomendaciones ordenadas por el puntaje ponderado
recommendations = sorted(zip(model.predict(X)[:, 0], filtered_shops), reverse=True)

# Imprimir las recomendaciones
for score, shop in recommendations:
    print(f"Score: {score}, Shop: {df_shop[df_shop['id_source'] == shop['id_source']]['name'].values[0]}")

Score: 0.9093362471715091, Shop: Valet Parkig
Score: 0.8866914087059915, Shop: The Attic
Score: 0.8861222501646722, Shop: Awesome Stay Inc.
Score: 0.88608150194264, Shop: Pueblo Viejo
Score: 0.8771299948179242, Shop: Live House
Score: 0.8749967738897408, Shop: The UPS Store
Score: 0.8738356836370739, Shop: Froman's Chicago Deep Dish Pizza
Score: 0.8730941646704495, Shop: Umami Burger - Eden
